# Mob Programming - Sept 6, 2016

At Chi Hack Night in Chicago, IL

Below is the Jupyter notebook from our mob programming session, with some annotations added after the fact, for clarity. 

The object was to compare these two datasets of complaints against police officers in the [chicago-police-data repository](https://github.com/invinst/chicago-police-data):
* Older data on [Citizen's Police Data Project](https://cpdb.co) (in repo, [/cpdb_complaints-cpd/](https://github.com/invinst/chicago-police-data/tree/master/cpdb_complaints-cpd))
* Data obtained recently in June 2016 by FOIA (in repo, [/complaints-cpd-june2016/](https://github.com/invinst/chicago-police-data/tree/master/complaints-cpd-june2016))

These two datasets contained some of the same information but were obtained at different times from the police department. For complaint cases overlapping between the two sets, we wanted to identify if the incident categorization changed over time. (The police department might recategorize incidents.)

In [60]:
# Import the pandas library for data manipulation, assign it the alias pd
import pandas as pd

In [61]:
# Read in the June data from a csv file
df = pd.read_csv('/Users/Jay/projects/chicago-police-data/complaints-cpd-june2016/june2016_all.csv')

/Users/Jay/miniconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (28,29,30,31,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# Print the first five rows to see what it looks like
df.head()

,Complaint_Number,Beat,Location_Code,Address,Street,Apartment,City_State_Zipcode,Incident_Datetime,Complaint_Date,Closed_Date,...,PO_Witness_Race,PO_Witness_Star,PO_Witness_Birth_Year,PO_Witness_Date_Appointed,Victim_Gender,Victim_Age,Victim_Race_Desc,Complainant_Gender,Complainant_Age,Complainant_Race_Desc
0,106213.0,1631.0,17,3700.0,N HARLEM AVE,NaN,CHICAGO IL 60634,2015-07-19 21:00:00,2015-07-20 00:00:00,2015-09-21 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,107899.0,611.0,17,7843.0,S HERMITAGE AVE,NaN,CHICAGO IL 6062,2015-08-16 20:20:00,2016-01-25 00:00:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,107901.0,1814.0,17,1622.0,N NORTH PARK AVE,NaN,CHICAGO IL 60614,2016-01-25 21:50:00,2016-01-26 00:00:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,108026.0,1722.0,04,4650.0,N PULASKI RD,NaN,CHICAGO IL,2013-10-14 00:01:00,2016-04-25 00:00:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,108026.0,1722.0,04,4650.0,N PULASKI RD,NaN,CHICAGO IL,2013-10-14 00:01:00,2016-04-25 00:00:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Print some information about each column: number of non-empty entries, data type
# Useful to have an idea and to diagnose potential later issues
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31597 entries, 0 to 31596
Data columns (total 40 columns):
Complaint_Number                   31597 non-null float64
Beat                               26935 non-null float64
Location_Code                      27149 non-null object
Address                            19684 non-null float64
Street                             20110 non-null object
Apartment                          1530 non-null object
City_State_Zipcode                 21396 non-null object
Incident_Datetime                  27151 non-null object
Complaint_Date                     27151 non-null object
Closed_Date                        24203 non-null object
Investigator_Name                  27753 non-null object
Investigator_Current_Assignment    27753 non-null float64
Investigator_Rank                  27753 non-null object
Investigator_Star                  19661 non-null float64
Investigator_Appointed_Date        26836 non-null object
Accused_Name                    

In [10]:
['Complaint_Number','Accused_Name','Accused_Complaint_Category']

['Complaint_Number', 'Accused_Name', 'Accused_Complaint_Category']

In [23]:
# Subsetting these three columns we're interested in and saving as a new dataframe
# .copy() creates a new copy, instead of just a view (reference to a subset of the bigger df dataframe)
# We do this because if you start modifying things in a view
# Otherwise, pandas will complain (it's an easy way to screw things up)
df_sm = df[['Complaint_Number','Accused_Name','Accused_Complaint_Category']].copy()

In [9]:
# Print first five rows, see what it looks like
df_sm.head()

,Complaint_Number,Accused_Name,Accused_Complaint_Category
0,106213.0,NaN,NaN
1,107899.0,NaN,NaN
2,107901.0,"GONZALES, ROBIN",NaN
3,108026.0,"BECKER, JOHN",NaN
4,108026.0,"MARKHAM, SEAN",NaN


In [12]:
# Import CPDB allegations dataset
df_allegations = pd.read_csv('/Users/Jay/projects/chicago-police-data/cpdb_complaints-cpd/Master Database 12-7-15 vF_allegations.csv')

/Users/Jay/miniconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
# Import CPDB officers dataset. these has names of officers and mapping to a CPDB internal ID number
df_officers = pd.read_csv('/Users/Jay/projects/chicago-police-data/cpdb_complaints-cpd/Master Database 12-7-15 vF_officers.csv')

In [14]:
df_officers.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9781 entries, 0 to 9780
Data columns (total 22 columns):
officer_id              9704 non-null float64
officer_name            9704 non-null object
officer_first           9704 non-null object
officer_last            9704 non-null object
gender                  8539 non-null object
race                    8539 non-null object
race_edit               9704 non-null object
appt_date               7758 non-null object
star                    8683 non-null float64
rank                    7659 non-null object
unit                    8042 non-null float64
birth_year              6872 non-null float64
Active (June 1 2015)    9704 non-null object
allegations_count       9704 non-null float64
discipline_count        9704 non-null float64
Unnamed: 15             0 non-null float64
Unnamed: 16             0 non-null float64
Unnamed: 17             0 non-null float64
Unnamed: 18             0 non-null float64
Unnamed: 19             0 non-null float

In [15]:
df_allegations.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56385 entries, 0 to 56384
Data columns (total 25 columns):
id                     56384 non-null float64
crid                   56384 non-null object
officer_id             48314 non-null float64
cat_id                 42779 non-null object
category               56384 non-null object
allegation_name        56384 non-null object
recc_finding           6831 non-null object
recc_outcome           16551 non-null float64
final_finding          33062 non-null object
final_outcome          28144 non-null float64
finding_edit           56384 non-null object
result                 56384 non-null object
outcome_edit           56384 non-null object
value                  56384 non-null object
beat                   27731 non-null float64
location               28585 non-null object
add1                   21125 non-null float64
add2                   21613 non-null object
city                   21887 non-null object
incident_date          28588 no

In [17]:
df_allegations.head()

,id,crid,officer_id,cat_id,category,allegation_name,recc_finding,recc_outcome,final_finding,final_outcome,...,location,add1,add2,city,incident_date,start_date,end_date,investigator_id,final_outcome_class,Unnamed: 24
0,1.0,280573,2104.0,03D,First Amendment and Illegal Arrest,Illegal Arrest / False Arrest,NaN,NaN,NS,600.0,...,NaN,NaN,NaN,NaN,NaN,2002-05-14,2002-10-30,NaN,not-sustained,NaN
1,2.0,281329,2104.0,05D,Arrest/Lock-up Procedures,Excessive Force / Off Duty - No Injury,NaN,NaN,UN,600.0,...,NaN,NaN,NaN,NaN,NaN,2002-06-08,2004-01-05,NaN,not-sustained,NaN
2,3.0,285275,2104.0,03D,First Amendment and Illegal Arrest,Illegal Arrest / False Arrest,NaN,NaN,EX,600.0,...,NaN,NaN,NaN,NaN,NaN,2002-10-25,2003-08-13,NaN,not-sustained,NaN
3,4.0,286312,2104.0,03B,First Amendment and Illegal Arrest,Improper Search Of Person,NaN,NaN,EX,600.0,...,NaN,NaN,NaN,NaN,NaN,2002-12-11,2003-05-01,NaN,not-sustained,NaN
4,5.0,286958,2104.0,03C,First Amendment and Illegal Arrest,Search Of Premise Without Warrant,NaN,NaN,UN,600.0,...,NaN,NaN,NaN,NaN,NaN,2003-01-11,2003-06-27,NaN,not-sustained,NaN


In [20]:
# We're going to subset on the rows we care about
df_allegations_sm = df_allegations[['crid', 'cat_id','category','allegation_name']]

In [21]:
df_allegations_sm.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56385 entries, 0 to 56384
Data columns (total 4 columns):
crid               56384 non-null object
cat_id             42779 non-null object
category           56384 non-null object
allegation_name    56384 non-null object
dtypes: object(4)
memory usage: 1.7+ MB


In [24]:
# We're going to want to df_sm and df_allegations_sm by complaint number
# From earlier, the June 2016 data had complaint numbers like 106213.0
# It thinks they're floats! But we want a string that's like '106213' to match df_allegations_sm
# So we're going to cast as int to round off the .0 and then cast as string
# Below we're using .apply, which lets you specify any arbitrary function
# The lambda x is how you write anonymous functions in Python

df_sm['Complaint_Number'] = df_sm['Complaint_Number'].apply(lambda x: str(int(x)))
df_sm.head()

# the smarter way to do this would be to use built-in pandas cast, e.g.
#      df_sm['Complaint_Number'].astype(int).astype(str)

,Complaint_Number,Accused_Name,Accused_Complaint_Category
0,106213,NaN,NaN
1,107899,NaN,NaN
2,107901,"GONZALES, ROBIN",NaN
3,108026,"BECKER, JOHN",NaN
4,108026,"MARKHAM, SEAN",NaN


In [26]:
# Here we do an inner join. This will drop rows where the complaint number doesn't exist for both dfs
df_merged = pd.merge(df_sm,df_allegations_sm,left_on='Complaint_Number', right_on='crid',how='inner')

In [27]:
df_merged.head()

,Complaint_Number,Accused_Name,Accused_Complaint_Category,crid,cat_id,category,allegation_name
0,1039179,"YANEZ, JUAN","05J-""U"" CONVERTED TO C.R. (RECORDS KEEPING ONL...",1039179,05J,Arrest/Lock-up Procedures,"U Converted To C.R. (Records Keeping Only, Ini..."
1,1039179,"YANEZ, JUAN","05J-""U"" CONVERTED TO C.R. (RECORDS KEEPING ONL...",1039179,05J,Arrest/Lock-up Procedures,"U Converted To C.R. (Records Keeping Only, Ini..."
2,1039179,"YANEZ, JUAN","05J-""U"" CONVERTED TO C.R. (RECORDS KEEPING ONL...",1039179,05J,Arrest/Lock-up Procedures,"U Converted To C.R. (Records Keeping Only, Ini..."
3,1039179,"YANEZ, JUAN","05J-""U"" CONVERTED TO C.R. (RECORDS KEEPING ONL...",1039179,05J,Arrest/Lock-up Procedures,"U Converted To C.R. (Records Keeping Only, Ini..."
4,1039179,"YANEZ, JUAN","05J-""U"" CONVERTED TO C.R. (RECORDS KEEPING ONL...",1039179,05J,Arrest/Lock-up Procedures,"U Converted To C.R. (Records Keeping Only, Ini..."


In [29]:
# We're going to drop duplicate rows
# In June 2016 we get duplicates from there being multiple officers involved in the same incident

df_merged_clean = df_merged.drop_duplicates()
df_merged_clean.head()

,Complaint_Number,Accused_Name,Accused_Complaint_Category,crid,cat_id,category,allegation_name
0,1039179,"YANEZ, JUAN","05J-""U"" CONVERTED TO C.R. (RECORDS KEEPING ONL...",1039179,05J,Arrest/Lock-up Procedures,"U Converted To C.R. (Records Keeping Only, Ini..."
6,1039179,"CHATYS, MARTIN","05J-""U"" CONVERTED TO C.R. (RECORDS KEEPING ONL...",1039179,05J,Arrest/Lock-up Procedures,"U Converted To C.R. (Records Keeping Only, Ini..."
12,1053497,"COLEMAN, MARVIN",10V-INVENTORY PROCEDURES,1053497,10V,Operation/Personnel Violations,Inventory Procedures
13,1053497,"EDWARDS, TRACEY",10V-INVENTORY PROCEDURES,1053497,10V,Operation/Personnel Violations,Inventory Procedures
14,1053502,"COLLIER, DWAYNE",03C-SEARCH OF PREMISE/VEHICLE WITHOUT WARRANT,1053502,03C,First Amendment and Illegal Arrest,Search Of Premise Without Warrant


In [ ]:
# We can see that Accused_Complaint_Category from June 2016 dataset has a bunch of extra info
# but cat_id from CPDB only has code
# We're going to try to use string subset comparison, if the cat_id entry is `in` other entry

# (Disclaimer, this next bit is wrong)

In [31]:
# Here we try to use .isin() to generate a column of booleans, sum to get number of trues
df_merged_clean['cat_id'].isin(df_merged_clean['Accused_Complaint_Category']).sum()

1038

In [34]:
# Here we use ~ to negate the booleans
(~df_merged_clean['cat_id'].isin(df_merged_clean['Accused_Complaint_Category'])).sum()

5697

In [62]:
# The above tells us that we have 1038 matches, and 5697 differences
# That seems like a lot of differences! 
# We got rid of the code that looked at what this did, but it's actually just catching NAs 
# Also we did it backwards,    x.is(y) checks if y is in x, it does *not* check if x is in y


In [54]:
# Here we're dropping rows if they have NAs in either of the two specified columns
# axis = 0 means we want to drop rows
df_merged_clean_nnans= df_merged_clean.dropna(axis=0,subset=['cat_id','Accused_Complaint_Category'])

In [42]:
# Here we were investigating if stripping whitespace might fix the earlier problem with isin()
# .strip() will strip trailing and leading whitespace
df_merged_clean_nnans[(df_merged_clean_nnans['cat_id'].str.strip().isin(df_merged_clean_nnans['Accused_Complaint_Category']))]

,Complaint_Number,Accused_Name,Accused_Complaint_Category,crid,cat_id,category,allegation_name


In [55]:
# Since we were messing up the .isin() earlier (we were getting it backwards),
# we decided to use apply instead and make a lambda function that does what we want
# here, axis=1 says we're applying this function across columns (so we're passing the function a row)
# this gives us a column of True if cat_id is in Accused_Complaint_Category
df_bool_compare = df_merged_clean_nnans.apply(lambda row: str(row['cat_id']) in str(row['Accused_Complaint_Category']),axis=1)

In [57]:
# Here we get the dataframe of our mismatches (we're negating the matches)
df_final_mismatch= df_merged_clean_nnans[~df_bool_compare]

In [59]:
# This gives us the number of rows and columns. so we have 125 mismatches!
df_final_mismatch.shape

(125, 7)

# Post session output

I'm just writing some additional code so we can get some results from this out of it

In [68]:
df_final_mismatch.head()

,Complaint_Number,Accused_Name,Accused_Complaint_Category,crid,cat_id,category,allegation_name
26,1053519,"SOSA, JESSICA",03B-SEARCH OF PERSON WITHOUT WARRANT,1053519,05N,Arrest/Lock-up Procedures,Unnecessary Physical Contact / Off Duty - No I...
39,1053524,"SKIBINSKI, CHRISTINE","05B-ARRESTEE - AFTER ARREST, PRIOR TO LOCKUP",1053524,05N,Arrest/Lock-up Procedures,Unnecessary Physical Contact / Off Duty - No I...
389,1053676,"FOSTER, BELINDA",05A-ARRESTEE - DURING ARREST,1053676,05B,Arrest/Lock-up Procedures,Excessive Force / On Duty - No Injury
390,1053676,"DUCK, MICHAEL","05B-ARRESTEE - AFTER ARREST, PRIOR TO LOCKUP",1053676,05A,Arrest/Lock-up Procedures,Excessive Force / On Duty - Injury
392,1053676,"STEELE, ROBERTA","05B-ARRESTEE - AFTER ARREST, PRIOR TO LOCKUP",1053676,05A,Arrest/Lock-up Procedures,Excessive Force / On Duty - Injury


In [70]:
print('Number of CRIDs in CPDB: ', df_allegations['crid'].nunique())
print('Number of CRIDs in June 2016: ', df['Complaint_Number'].nunique())
print('Number of overlapping CRIDs: ', df_merged['crid'].nunique())
print('')
print('Number of rows for overlapping CRIDs with both non-null cat codes: ', df_merged_clean_nnans['crid'].shape[0])
print('')
print('Number of overlapping CRIDs with both non-null cat codes: ', df_merged_clean_nnans['crid'].nunique())
print('Number of mismatches: ', df_final_mismatch['crid'].nunique())

Number of CRIDs in CPDB:  39591
Number of CRIDs in June 2016:  16531
Number of overlapping CRIDs:  2729

Number of rows for overlapping CRIDs with both non-null cat codes:  5505

Number of overlapping CRIDs with both non-null cat codes:  2293
Number of mismatches:  67


It looks like we were still having some multiple rows because we kept the Accused_Name and there are multiple accused officers for each CRID. So there were 67 unique CRIDs between the two sets where the categories changed. 

However, this doesn't account for complaints where it was either blank for one but not the other. 

In [73]:
# Saving df_final_mismatch to csv
df_merged.to_csv('june2016-cpdb_merged.csv',index=False)
df_final_mismatch.to_csv('june2016-cpdb_category_mismatch.csv',index=False)